In [12]:
import numpy as np
import pandas as pd
import torch
from typing import Literal, Callable
from sklearn.metrics import classification_report


class LogisticRegressionRBF(torch.nn.Module):

    def __init__(self, x_basis: torch.Tensor, rbf: Literal['linear', 'gaussian'] = 'gaussian',
                 print_function: Callable = print):
        """
        :param x_basis: centers of basis functions
        :param rbf: type of rbf function. Available: ['linear', 'gaussian']
        """
        super(LogisticRegressionRBF, self).__init__()

        self.w = torch.nn.Linear(x_basis.shape[0], 1)
        self.rbf = rbf
        self.x_basis = x_basis
        self.print = print_function
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x: torch.Tensor = None, phi_matrix: torch.Tensor = None) -> torch.Tensor:
        """
        Returns a "probability" (confidence) of class 1

        :param x: 2D array
        :param phi_matrix: 2D array
        :return: 1D array
        """
        if phi_matrix is None:
            phi_matrix = self.make_phi_matrix(x)

        return self.sigmoid(self.w(phi_matrix))

    def make_phi_matrix(self, x: torch.Tensor) -> torch.Tensor:
        """
        Returns k x n array with calculated phi(x_i, x_basis_j)

        :param x: Array k x m dimensional. k different x_i and m features
        """
        n = x.shape[0]
        m = x.shape[1]
        k = n * m  # Количество всех элементов

        repd_row_matrix = torch.tile(x, (1, n)).reshape(n ** 2, m)
        repd_whole_matrix =  torch.tile(x, (n, 1))
        phi = (repd_row_matrix - repd_whole_matrix) ** 2
        phi = phi.sum(axis=1).reshape(n, n)

        if self.rbf == 'linear':
            phi = phi ** 0.5
            phi = phi / phi.max()
        elif self.rbf == 'gaussian':
            phi = torch.exp(-phi)
        elif self.rbf == 'multiquadratic':
            phi = (1 + phi) ** 0.5
            phi = phi / phi.max()

        return phi

    def fit(self, x, y, epochs = 1):

        print_epochs = np.unique(np.geomspace(1, epochs + 1, 15, dtype=int))

        phi_matrix = self.make_phi_matrix(x)
        optimizer = torch.optim.Adam(self.parameters())
        criterion = torch.nn.CrossEntropyLoss()
        loss = torch.nn.BCELoss()

        for epoch in range(1, epochs + 1):
            optimizer.zero_grad()
            output = loss(self.forward(x, phi_matrix).flatten(), y)
            output.backward()
            optimizer.step()

            with torch.no_grad():
                if epoch + 1 in print_epochs:
                    print(f'Epoch: {epoch: 5d} | CrossEntropyLoss: {output.item(): 0.5f}')

        return self

    def metrics_tab(self, x, y):
        y_prob = self.forward(x)
        y_pred = (y_prob > 0.5) * 1
        output = classification_report(y, y_pred, output_dict=True)
        return output

In [13]:
from sklearn.datasets import make_classification

x, y = make_classification(n_samples=500, n_features=4, random_state=1)
x = torch.FloatTensor(x[:, :2])
y = torch.FloatTensor(y)

In [14]:
model = LogisticRegressionRBF(x)
model.fit(x, y, 100)

Epoch:     1 | CrossEntropyLoss:  0.79643
Epoch:     2 | CrossEntropyLoss:  0.76653
Epoch:     4 | CrossEntropyLoss:  0.70968
Epoch:     6 | CrossEntropyLoss:  0.65691
Epoch:     9 | CrossEntropyLoss:  0.58555
Epoch:    12 | CrossEntropyLoss:  0.52351
Epoch:    18 | CrossEntropyLoss:  0.42544
Epoch:    26 | CrossEntropyLoss:  0.33831
Epoch:    36 | CrossEntropyLoss:  0.27483
Epoch:    51 | CrossEntropyLoss:  0.22654
Epoch:    71 | CrossEntropyLoss:  0.19621
Epoch:   100 | CrossEntropyLoss:  0.17454


LogisticRegressionRBF(
  (w): Linear(in_features=500, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [15]:
import pandas as pd
pd.DataFrame(model.metrics_tab(x, y)).T

,precision,recall,f1-score,support
0.0,0.943320,0.939516,0.941414,248.000
1.0,0.940711,0.944444,0.942574,252.000
accuracy,0.942000,0.942000,0.942000,0.942
macro avg,0.942016,0.941980,0.941994,500.000
weighted avg,0.942005,0.942000,0.941999,500.000


In [46]:
y_prob = model(x).detach().cpu().numpy().flatten()

In [47]:
y_pred = (y_prob > 0.6) * 1

In [58]:
from numbers import Real
import numpy as np


def tpr(y_true: np.array, y_pred: np.array) -> Real:
    """
    Return True Positive Rate. TPR = TP / P = TP / (TP + FN).

    .. note:: if P == 0, then TPR == 0

    :param y_true: array with true values of binary classification
    :param y_pred: array with prediction values of binary classification
    :return:
    """
    y_true = np.array(y_true, dtype=int)
    y_pred = np.array(y_pred, dtype=int)
    tp = sum((y_true == y_pred) & (y_true == 1))
    p = sum(y_true == 1)
    return tp / max(p, 1)

def fpr(y_true: np.array, y_pred: np.array) -> Real:
    """
    Return False Positive Rate. FPR = FP / N = FP / (FP + TN).

    .. note:: if N == 0, then FPR == 0

    :param y_true: array with true values of binary classification
    :param y_pred: array with prediction values of binary classification
    :return:
    """
    y_true = np.array(y_true, dtype=int)
    y_pred = np.array(y_pred, dtype=int)
    fp = sum((y_true != y_pred) & (y_true == 0))
    n = sum(y_true == 0)
    return fp / max(n, 1)


def roc_auc_score(y_true: np.array, y_prob: np.array) -> Real:
    """
    Compute area under the ROC curve from predictions y_prob.

    :param y_true: array with true values of binary classification
    :param y_prob: array of probabilities of confidence of belonging to the 1st class
    :return: float number of area
    """



In [59]:
tpr(y.flatten(), y_pred)

0.9285714285714286

In [55]:
y.flatten()


tensor([0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1.,
        1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1.,
        1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0.,
        0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1.,
        0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1.,
        0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1.,
        0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1.,
        0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0.,
        1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1.,
        0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
        0., 0., 1., 0., 1., 1., 0., 0., 

In [50]:
from sklearn.metrics import roc_curve

In [35]:
roc_curve(y, y_prob)

(array([0.        , 0.        , 0.        , 0.00403226, 0.00403226,
        0.01612903, 0.01612903, 0.02016129, 0.02016129, 0.02419355,
        0.02419355, 0.02822581, 0.02822581, 0.03225806, 0.03225806,
        0.04032258, 0.04032258, 0.04435484, 0.04435484, 0.06451613,
        0.06451613, 0.08467742, 0.08467742, 0.08870968, 0.08870968,
        0.09677419, 0.09677419, 0.10080645, 0.10080645, 0.13709677,
        0.13709677, 0.16129032, 0.16129032, 0.16532258, 0.16532258,
        0.25806452, 0.25806452, 0.44354839, 0.44354839, 0.5483871 ,
        0.5483871 , 0.91129032, 0.91129032, 1.        ]),
 array([0.        , 0.00396825, 0.68253968, 0.68253968, 0.81349206,
        0.81349206, 0.88888889, 0.88888889, 0.89285714, 0.89285714,
        0.8968254 , 0.8968254 , 0.9047619 , 0.9047619 , 0.92857143,
        0.92857143, 0.93253968, 0.93253968, 0.94444444, 0.94444444,
        0.95238095, 0.95238095, 0.95634921, 0.95634921, 0.96031746,
        0.96031746, 0.96428571, 0.96428571, 0.97222222, 0.

In [28]:
y_pred

tensor([0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,
        1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
        0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0,
        0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1,
        1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1,
        1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
        1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
        0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
        1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0,
        0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1,
        1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,